In [1]:
# import tensorflow.compat.v1 as tf
# from tensorflow.compat.v1 import keras
from os import listdir
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imsave, imshow
import numpy as np
import pandas as pd
import math
from scipy.signal import convolve2d
from skimage import img_as_ubyte, img_as_float
from skimage.transform import resize
# from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
# from keras import Sequential
# from keras import layers
import copy
from skimage.color import rgb2gray
from sklearn.metrics import mean_squared_error as MSE
import gc
import os.path
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import peak_signal_noise_ratio as PSNR
gc.enable()

# tf.keras.backend.clear_session()  # For easy reset of notebook state.
# sess = tf.Session() 
# keras.backend.set_session(sess)

from lib import make_directory

def make_ker(ker_len, ker_angle):
#     h = ker_len * 2
    h = ker_len
    ker_len = ker_len // 2
    ker = np.zeros((h, h), dtype='float')
    k = -math.tan(ker_angle)
    b = (1 - k) * ker_len
    if abs(abs(ker_angle * 180/math.pi) - 90) > 10:
        for x in range(h):
            y = round(k * x + b)
            y = int((y if y >= 0 else 0) if y <= h-1 else h-1)
            if (y == 0 or y == h - 1):
                continue
            ker[y, x] = 1
    else:
        for y in range(h):
            ker[y, ker_len] = 1 
    ret_value = ker/ker.sum()
    if np.isnan(np.sum(ret_value)):
        return []
    else:
        return ret_value

In [2]:
import random
from skimage.transform import SimilarityTransform
from skimage.transform import warp
from skimage.util import random_noise
from scipy.ndimage import gaussian_filter

# IMG_SIZE = 200
BLUR_LEN = 5
# IMG_SIZE = None
# OUT_SIZE = 600
BATCH_SIZE = 8
EPS = 10e-7

def rotate_transform_matrix(transform):
    """Rotate matrix so it can be applied to row:col coordinates."""
    matrix = transform.params[(1, 0, 2), :][:, (1, 0, 2)]
    return type(transform)(matrix)

def prepare_img(img):
    image = copy.copy(img)
    if len(image.shape) == 2:
        image = np.stack((image, image, image))
        image = np.transpose(image, axes=(1,2, 0))
    for channel in range(3):
        image[:,:,channel] = (image[:,:,channel] - np.mean(image[:,:,channel])) / (np.std(image[:,:,channel]) + EPS)
    if (image.shape[2] == 4):
        image = image[:,:,:3]
    h, w = image.shape[:2]
    return image

def prepare_img1(img):
    image = copy.copy(img)
    if len(image.shape) == 2:
        image = np.stack((image, image, image))
        image = np.transpose(image, axes=(1,2, 0))
    if (image.shape[2] == 4):
        image = image[:,:,:3]
    return img_as_float(image)

# train_generator = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.1)
# train_generator.fit(X, augment=True, seed=42)


In [3]:
import random

In [4]:
IMG_SIZE = 1000
BLUR_LEN = 5
KER = make_ker(BLUR_LEN, 0)
global_idx = 0

def generate_pics(paths, amount_on_picture, dir_to_save = './', break_on = 0):
    global global_idx
    make_directory(dir_to_save)
    make_directory(dir_to_save + 'blurred')
    make_directory(dir_to_save + 'sharp')
    for p in paths:
        try:
            cur_img = prepare_img1(imread(p))
            blurred = convolve_img(cur_img)
            noised = random_noise(blurred, var=0.001)
            h, w = cur_img.shape[:2]
            if (h > IMG_SIZE and w > IMG_SIZE):
                for idx in range(amount_on_picture):
                    y = random.randrange(h - IMG_SIZE)
                    x = random.randrange(w - IMG_SIZE)
                    plt.imsave(dir_to_save + 'blurred/img_' + str(global_idx) + '.png', noised[y : y + IMG_SIZE, x : x + IMG_SIZE])
                    plt.imsave(dir_to_save + 'sharp/img_' + str(global_idx) + '.png', cur_img[y : y + IMG_SIZE, x : x + IMG_SIZE])
                    global_idx += 1
            if (break_on != 0):
                if (break_on == global_idx):
                    return
        except Exception as e: # work on python 2.x
            print('caught ex', str(e))

# def generate_pics_from_two(paths_X, paths_y, amount_on_picture):
#     global_idx = 0
#     for p in range(len(paths_X)):
#         blurred = prepare_img1(imread(paths_X[p]))
#         cur_img = prepare_img1(imread(paths_y[p]))
#         h, w = cur_img.shape[:2]
#         for idx in range(amount_on_picture):
#             y = random.randrange(h - IMG_SIZE)
#             x = random.randrange(w - IMG_SIZE)
#             plt.imsave('pict/b_test/test_' + str(global_idx) + '.png', blurred[y : y + IMG_SIZE, x : x + IMG_SIZE])
#             plt.imsave('pict/s_test/test_' + str(global_idx) + '.png', cur_img[y : y + IMG_SIZE, x : x + IMG_SIZE])
#             global_idx += 1
#     return

def convolve_img(image):
    # ker = KER
        # ker_len = BLUR_LEN
        ker_len = 5
        pad = ker_len // 2
        # ker = make_ker(ker_len, 0)
        # ker = make_ker(ker_len, random.uniform(0, np.pi))
        ker = make_ker(random.randint(3, 15), random.uniform(0, np.pi))
        img = image
        if len(img.shape) == 2:
            img = np.stack((image, image, image))
            img = np.transpose(image, axes=(1,2, 0))
        # for channel in range(3):
            # image[:,:,channel] = (image[:,:,channel] - np.mean(image[:,:,channel])) / (np.std(image[:,:,channel]) + EPS)
        r = convolve2d(np.pad(img[:,:,0], pad, 'edge'), ker, mode='valid'); #r /= r.max()
        r = gaussian_filter(r, sigma=0.7)
        g = convolve2d(np.pad(img[:,:,1], pad, 'edge'), ker, mode='valid'); #g /= g.max()
        g = gaussian_filter(g, sigma=0.7)
        b = convolve2d(np.pad(img[:,:,2], pad, 'edge'), ker, mode='valid'); #b /= b.max()
        b = gaussian_filter(b, sigma=0.7)
        image = np.stack((r, g, b))
        image = np.clip(image, 0., 1.)
        image = np.transpose(image, axes=(1,2, 0))
        return image

In [10]:
# generate_pics_from_two(X, y, 1)
generate_pics(y, 1, dir_to_save= 'pict/random_params/', break_on=20)

In [6]:
%%time
# train_dir = '../course/motion_blurred/'
# train_dir = 'b/'
y_dir = '../../Stuff/Images/google_big/g/'
# y_dir = 'pict/s_/'
# y_dir = 's/'
# fnames_X = listdir(train_dir)
fnames_y = os.listdir(y_dir)
fnames_y.sort()
# X = list([train_dir + item for item in fnames_X])
y = list([y_dir + item for item in fnames_y])

CPU times: user 6.31 ms, sys: 26.9 ms, total: 33.2 ms
Wall time: 94 ms


In [5]:
from metrics import calc_metrics

In [3]:
pre = '../../Stuff/Images/generated/val_l5a0/'
sharp_blurred = calc_metrics(pre + 'sharp', pre + 'blurred')
sharp_blurred

{'PSNR': 27.996653160101214, 'SSIM': 0.7770896768719642}

In [4]:
sharp_base = calc_metrics(pre + 'sharp', pre + 'pred_base')
sharp_base

{'PSNR': 28.10145503405277, 'SSIM': 0.7810993018365616}

In [5]:
sharp_last = calc_metrics(pre + 'sharp', pre + 'pred_last')
sharp_last

{'PSNR': 28.681493723878155, 'SSIM': 0.8045185526322394}

In [6]:
sharp_best = calc_metrics(pre + 'sharp', pre + 'pred_best')
sharp_best

{'PSNR': 28.621626154591645, 'SSIM': 0.8023219266443048}

In [7]:
def compare_dir_to_sharp(dir_to_check = './pict/d', dir_sharp = './pict/sharp'):
    fin_dict = dict()
    directories = listdir(dir_to_check)

    files = listdir(os.path.join(dir_to_check, directories[0]))

    l = len(files)

    for idx in range(l):
        sharp = prepare_img1(imread(os.path.join(dir_sharp, files[idx])))
        fin_dict[files[idx]] = dict()
        for d in directories:
            metrics = dict()
            pred = prepare_img1(imread(os.path.join(dir_to_check, d, files[idx]))[:IMG_SIZE, :IMG_SIZE])
            try:
                metrics['PSNR'] = PSNR(sharp, pred)
                metrics['SSIM'] = SSIM(sharp, pred, multichannel=True)
                fin_dict[files[idx]][d] = metrics
            except Exception as e:
                print(str(e), 'image ', files[idx], ', directory ', d)

    return fin_dict

In [8]:
%%time
fin_dict = compare_dir_to_sharp()

Input images must have the same dimensions. image  img_3.png , directory  l07
Input images must have the same dimensions. image  img_3.png , directory  base
Input images must have the same dimensions. image  img_3.png , directory  blurred
Input images must have the same dimensions. image  img_3.png , directory  l05
Input images must have the same dimensions. image  img_3.png , directory  l03
Input images must have the same dimensions. image  img_3.png , directory  l09
Input images must have the same dimensions. image  img_3.png , directory  l11
CPU times: user 1min 19s, sys: 9.03 s, total: 1min 28s
Wall time: 1min 29s


In [9]:
fin_dict

{'img_2.png': {'l07': {'PSNR': 22.048444166475818, 'SSIM': 0.7707385724286953},
  'base': {'PSNR': 22.0814215142806, 'SSIM': 0.7113280848504555},
  'blurred': {'PSNR': 22.790239793681234, 'SSIM': 0.7183136312011357},
  'l05': {'PSNR': 22.247714296004197, 'SSIM': 0.7826727711810536},
  'l03': {'PSNR': 22.12044428582177, 'SSIM': 0.7750542838857211},
  'l09': {'PSNR': 21.85208463868516, 'SSIM': 0.7646636019633797},
  'l11': {'PSNR': 21.398211579133246, 'SSIM': 0.761464035863554}},
 'img_1.png': {'l07': {'PSNR': 18.832768617833878, 'SSIM': 0.6681014066977594},
  'base': {'PSNR': 22.515142528324542, 'SSIM': 0.7585836752013998},
  'blurred': {'PSNR': 22.714696769188343, 'SSIM': 0.7626377439077606},
  'l05': {'PSNR': 18.813196064481595, 'SSIM': 0.6523205830693343},
  'l03': {'PSNR': 19.20178338968236, 'SSIM': 0.6679211538791057},
  'l09': {'PSNR': 18.440071512547973, 'SSIM': 0.6566420881788159},
  'l11': {'PSNR': 18.295504403696533, 'SSIM': 0.6390017891960732}},
 'img_4.png': {'l07': {'PSNR':

In [14]:
metric = 'SSIM'
max_params = dict()
for img_key in fin_dict:
    params = fin_dict[img_key]
    max_val = 0
    k = list(params.keys())
    max_key = k[0] if len(k) > 0 else 0
    max_params[img_key] = dict()
    for dir_key in params:
        if dir_key == 'blurred' or dir_key == 'base':
            max_params[img_key][dir_key] = params[dir_key]
        elif params[dir_key][metric] > params[max_key][metric]:
            print(params[dir_key][metric], params[max_key][metric])
            max_key = dir_key
    if (max_key in params.keys()):   
        max_params[img_key][max_key] = params[max_key]


max_params

0.7826727711810536 0.7707385724286953
0.5249240032353949 0.4811458667324606
0.5713109479734043 0.5561174739951777
0.5749437626355006 0.5713109479734043
0.18555297000197224 0.1526093528469206
0.19155270440002245 0.18555297000197224
0.7594835452396355 0.7255776045748515
0.7664875222301416 0.7594835452396355
0.6431989527065469 0.6424765397496119
0.6561649840292948 0.6431989527065469
0.7119705399397948 0.6736544615679746
0.6814661113597665 0.6596962946128113
0.8100182741239186 0.8065494389363038
0.8120303010764105 0.7818199943998417
0.2682086072974195 0.2390117965308595
0.8010652454827834 0.7990563656536325
0.8574550179544721 0.814889774985445
0.8173342276647332 0.8005547535290991
0.6499522540796051 0.6361740265211083
0.6521791698611062 0.6499522540796051
0.6579816637100396 0.6521791698611062


{'img_2.png': {'base': {'PSNR': 22.0814215142806, 'SSIM': 0.7113280848504555},
  'blurred': {'PSNR': 22.790239793681234, 'SSIM': 0.7183136312011357},
  'l05': {'PSNR': 22.247714296004197, 'SSIM': 0.7826727711810536}},
 'img_1.png': {'base': {'PSNR': 22.515142528324542,
   'SSIM': 0.7585836752013998},
  'blurred': {'PSNR': 22.714696769188343, 'SSIM': 0.7626377439077606},
  'l07': {'PSNR': 18.832768617833878, 'SSIM': 0.6681014066977594}},
 'img_4.png': {'base': {'PSNR': 19.11596099485838, 'SSIM': 0.5116330042660148},
  'blurred': {'PSNR': 19.91404065191115, 'SSIM': 0.5474098154435039},
  'l05': {'PSNR': 19.143465975030818, 'SSIM': 0.5249240032353949}},
 'img_10.png': {'base': {'PSNR': 18.62504614093614, 'SSIM': 0.540532022598535},
  'blurred': {'PSNR': 19.135895317744108, 'SSIM': 0.5570158746380436},
  'l03': {'PSNR': 18.786505181670343, 'SSIM': 0.5749437626355006}},
 'img_8.png': {'base': {'PSNR': 14.91875070494239, 'SSIM': 0.1858936308383589},
  'blurred': {'PSNR': 14.911149484596365, 

In [15]:
max_params

{'img_2.png': {'base': {'PSNR': 22.0814215142806, 'SSIM': 0.7113280848504555},
  'blurred': {'PSNR': 22.790239793681234, 'SSIM': 0.7183136312011357},
  'l05': {'PSNR': 22.247714296004197, 'SSIM': 0.7826727711810536}},
 'img_1.png': {'base': {'PSNR': 22.515142528324542,
   'SSIM': 0.7585836752013998},
  'blurred': {'PSNR': 22.714696769188343, 'SSIM': 0.7626377439077606},
  'l07': {'PSNR': 18.832768617833878, 'SSIM': 0.6681014066977594}},
 'img_4.png': {'base': {'PSNR': 19.11596099485838, 'SSIM': 0.5116330042660148},
  'blurred': {'PSNR': 19.91404065191115, 'SSIM': 0.5474098154435039},
  'l05': {'PSNR': 19.143465975030818, 'SSIM': 0.5249240032353949}},
 'img_10.png': {'base': {'PSNR': 18.62504614093614, 'SSIM': 0.540532022598535},
  'blurred': {'PSNR': 19.135895317744108, 'SSIM': 0.5570158746380436},
  'l03': {'PSNR': 18.786505181670343, 'SSIM': 0.5749437626355006}},
 'img_8.png': {'base': {'PSNR': 14.91875070494239, 'SSIM': 0.1858936308383589},
  'blurred': {'PSNR': 14.911149484596365, 